# Hussaini lab data (Axona) conversion to NWB

Here, I am going to test code for doing said conversions.

### NWB

To convert to NWB we will need a separate Interface class (e.g. AxonaRecordingExtractorInterface) for each data type / extractor. Those are then combined with an NWBConverter class for the Lab (e.g. HussainiLabNWBConverter), which exports to NWB. 

Note that for Milestone2, we might not need any of this, since spikeextractors has its own NWBconverters for extractor and sorting classes!

Adapted from https://github.com/catalystneuro/movshon-lab-to-nwb/blob/main/tutorials/blackrock_nwb_conversion_detailed.ipynb

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
set_file = filename + '.set'
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [3]:
!ls ../nwb-conversion-tools/nwb_conversion_tools -l

total 76
-rwxrwxrwx 1 sbuergers sbuergers    69 Apr  5 11:28 __init__.py
drwxrwxrwx 1 sbuergers sbuergers  4096 Apr 28 12:03 __pycache__
-rwxrwxrwx 1 sbuergers sbuergers  1404 Apr  5 11:28 auto_qc.py
-rwxrwxrwx 1 sbuergers sbuergers  1338 Apr  5 11:28 basedatainterface.py
-rwxrwxrwx 1 sbuergers sbuergers  2134 Apr  5 11:28 baseimagingextractorinterface.py
-rwxrwxrwx 1 sbuergers sbuergers  2812 Apr 28 11:58 baselfpextractorinterface.py
-rwxrwxrwx 1 sbuergers sbuergers  5385 Apr 28 11:58 baserecordingextractorinterface.py
-rwxrwxrwx 1 sbuergers sbuergers  2363 Apr  5 11:28 basesegmentationextractorinterface.py
-rwxrwxrwx 1 sbuergers sbuergers  3786 Apr 28 11:58 basesortingextractorinterface.py
-rwxrwxrwx 1 sbuergers sbuergers  5015 Apr 28 11:58 conversion_tools.py
drwxrwxrwx 1 sbuergers sbuergers  4096 Apr 28 11:58 datainterfaces
-rwxrwxrwx 1 sbuergers sbuergers  5100 Apr 28 11:58 json_schema_utils.py
-rwxrwxrwx 1 sbuergers sbuergers 10154 Apr  5 11:28 metafile.schema.json
-rwxrwxrwx 1 s

In [4]:
# Import modules

import random
import string
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np
import re
import datetime
import json
from jsonschema import validate, ValidationError

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface,  BaseDataInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from nwb_conversion_tools.json_schema_utils import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools.datainterfaces.interface_utils.brpylib import NsxFile
from nwb_conversion_tools import SpikeGLXRecordingInterface

In [5]:
# from basedatainterface.py
base_schema = get_base_schema(
    id_='metadata.schema.json',
    root=True,
    title='Metadata',
    description='Schema for the metadata',
    version="0.1.0",
)

In [6]:
base_schema

{'required': [],
 'properties': {},
 'type': 'object',
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'metadata.schema.json',
 'title': 'Metadata',
 'description': 'Schema for the metadata',
 'version': '0.1.0'}

In [7]:
glx = SpikeGLXRecordingInterface

In [8]:
glx.get_source_schema()

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [9]:
import inspect

inspect.signature(glx.__init__).parameters

mappingproxy({'self': <Parameter "self">,
              'file_path': <Parameter "file_path: Union[str, pathlib.Path, NoneType]">,
              'stub_test': <Parameter "stub_test: Union[bool, NoneType] = False">})

In [10]:
!ls ../spikeextractors/spikeextractors -l

total 224
-rwxrwxrwx 1 sbuergers sbuergers  1265 Mar 19 08:48 __init__.py
drwxrwxrwx 1 sbuergers sbuergers  4096 Apr 28 12:02 __pycache__
-rwxrwxrwx 1 sbuergers sbuergers 22834 Apr 27 20:11 baseextractor.py
-rwxrwxrwx 1 sbuergers sbuergers  9153 Apr 27 20:11 cacheextractors.py
drwxrwxrwx 1 sbuergers sbuergers  4096 Apr 20 21:40 example_datasets
-rwxrwxrwx 1 sbuergers sbuergers   105 Mar 19 08:48 exceptions.py
-rwxrwxrwx 1 sbuergers sbuergers 37347 Apr 27 20:11 extraction_tools.py
-rwxrwxrwx 1 sbuergers sbuergers  5952 Apr 28 11:50 extractorlist.py
drwxrwxrwx 1 sbuergers sbuergers  4096 Mar 22 09:47 extractors
-rwxrwxrwx 1 sbuergers sbuergers  6255 Apr 27 20:11 multirecordingchannelextractor.py
-rwxrwxrwx 1 sbuergers sbuergers  8547 Apr 27 20:11 multirecordingtimeextractor.py
-rwxrwxrwx 1 sbuergers sbuergers  5459 Apr 27 20:11 multisortingextractor.py
-rwxrwxrwx 1 sbuergers sbuergers 41337 Apr 27 20:11 recordingextractor.py
-rwxrwxrwx 1 sbuergers sbuergers 30652 Apr 27 20:11 sortingextr

In [11]:
from spikeextractors.extractors.neoextractors import AxonaRecordingExtractor

In [12]:
are = AxonaRecordingExtractor(filename=filename)
print('Number of channels:', are.get_num_channels())
print('Channel groups:', are.get_channel_groups())

Number of channels: 16
Channel groups: [0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3]


Parse .set file for metadata to include in metadata.

In [13]:
def parse_generic_header(filename, params):
    """
    Given a binary file with phrases and line breaks, enters the
    first word of a phrase as dictionary key and the following
    string (without linebreaks) as value. Returns the dictionary.
    
    INPUT
    filename (str): .set file path and name.
    params (list or set): parameter names to search for. 
    
    OUTPUT
    header (dict): dictionary with keys being the parameters that
                   were found & values being strings of the data.
                   
    EXAMPLE
    parse_generic_header('myset_file.set', ['experimenter', 'trial_time'])
    """
    header = {}
    params = set(params)
    with open(filename, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                break
            line = bin_line.decode('cp1252').replace('\r\n', '').replace('\r', '').strip()
            parts = line.split(' ')
            key = parts[0]
            if key in params:
                header[key] = ' '.join(parts[1:])
            
    return header

In [14]:
params_of_interest = [
    'experimenter', 
    'comments',
    'duration', 
    'sw_version',
    'tracker_version',
    'stim_version',
    'audio_version'
]

In [15]:
parse_generic_header(set_file, params_of_interest)

{'experimenter': 'Abid',
 'comments': '',
 'duration': '600.00625',
 'sw_version': '1.2.2.16',
 'tracker_version': '0',
 'stim_version': '1',
 'audio_version': '0'}

In [16]:
def read_iso_datetime(set_file):
    """ 
    Creates datetime object (y, m, d, h, m, s) from .set file header 
    """
    with open(set_file, 'r', encoding='cp1252') as f:
        for line in f:
            if line.startswith('trial_date'):
                date_string = re.findall(r'\d+\s\w+\s\d{4}$', line)[0]
            if line.startswith('trial_time'):
                time_string = line[len('trial_time')+1::].replace('\n', '')

    return datetime.datetime.strptime(date_string + ', ' + time_string, \
        "%d %b %Y, %H:%M:%S").isoformat()

In [17]:
class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['filename'],
            'properties': {
                'filename': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
        
    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""        
                
        # Extract information for specific parameters from .set file
        params_of_interest = [
            'experimenter', 
            'comments',
            'duration', 
            'sw_version',
            'tracker_version',
            'stim_version',
            'audio_version'
        ]
        set_file = self.source_data['filename']+'.set'
        par = parse_generic_header(set_file, params_of_interest)
        
        # Extract information from AxonaRecordingExtractor
        elec_group_names = self.recording_extractor.get_channel_groups()
        unique_elec_group_names = set(elec_group_names)
        
        # Add available metadata
        metadata = super().get_metadata()
        metadata['NWBFile'] = dict(
            session_start_time=read_iso_datetime(set_file),
            session_description=par['comments'],
            session_duration=par['duration']+'s',
            experimenter=[par['experimenter']]
        )
        
        metadata['Ecephys'] = dict(
            Device=[
                dict(
                    name="Axona",
                    description="Axona DacqUSB, sw_version={}".format(par['sw_version']),
                    manufacturer="Axona"
                ),
            ],
            ElectrodeGroup=[
                dict(
                    name=f'Group{group_name}',
                    location='',
                    device='Axona',
                    description=f"Group {group_name} electrodes.",
                )
                for group_name in unique_elec_group_names
            ],
            Electrodes=[
                dict(
                    name='group_name',
                    description="The name of the ElectrodeGroup this electrode is a part of.",
                    data=[f"Group{x}" for x in elec_group_names]
                )
            ],
            ElectricalSeries=dict(
                name='ElectricalSeries',
                description="Raw acquisition traces."
            )
        )
  
        return metadata

In [38]:
### OLD BEFORE 14h 28.04.2021
"""Authors: Steffen Buergers"""
import re
import datetime
import spikeextractors as se

from pynwb.ecephys import ElectricalSeries
from nwb_conversion_tools.utils import get_schema_from_hdmf_class
from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface


def parse_generic_header(filename, params):
    """
    Given a binary file with phrases and line breaks, enters the
    first word of a phrase as dictionary key and the following
    string (without linebreaks) as value. Returns the dictionary.

    INPUT
    filename (str): .set file path and name.
    params (list or set): parameter names to search for.

    OUTPUT
    header (dict): dictionary with keys being the parameters that
                   were found & values being strings of the data.

    EXAMPLE
    parse_generic_header('myset_file.set', ['experimenter', 'trial_time'])
    """
    header = {}
    params = set(params)
    with open(filename, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                break
            line = bin_line.decode('cp1252').replace('\r\n', '').\
                replace('\r', '').strip()
            parts = line.split(' ')
            key = parts[0]
            if key in params:
                header[key] = ' '.join(parts[1:])

    return header


def read_iso_datetime(set_file):
    """
    Creates datetime object (y, m, d, h, m, s) from .set file header
    and converts it to ISO 8601 format
    """
    with open(set_file, 'r', encoding='cp1252') as f:
        for line in f:
            if line.startswith('trial_date'):
                date_string = re.findall(r'\d+\s\w+\s\d{4}$', line)[0]
            if line.startswith('trial_time'):
                time_string = line[len('trial_time')+1::].replace('\n', '')

    return datetime.datetime.strptime(date_string + ', ' + time_string,
                                      "%d %b %Y, %H:%M:%S").isoformat()


class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor"""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['filename'],
            'properties': {
                'filename': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
    
    def get_metadata_schema(self):
        metadata_schema = super().get_metadata_schema()
        
        # Update Ecephys metadata
        Electrodes = {
            "required": [
                "name",
                "description",
                "data"
            ],
            "properties": {
                "name": {
                    "description": "Electrode group name this electrode is a part of.",
                    "type": "string"
                },
                "description": {
                    "description": "Description of this electrode group",
                    "type": "string"
                },
                "data": {
                    "description": "Electrode group name for each electrode.",
                    "type": "array",
                }
            },
            "type": "array",
            "additionalProperties": False,
            "tag": "Electrodes"
        }
        
        metadata_schema['properties']['Ecephys']['properties']["definitions"].update(
            dict(
                Electrodes=Electrodes,
                ElectricalSeries=get_schema_from_hdmf_class(ElectricalSeries),
            )
        )      

        metadata_schema['properties']['Ecephys']['required'].extend(
            ['Electrodes', 'ElectricalSeries']
        )
        metadata_schema['properties']['Ecephys']['properties'].update(
            dict(
                Electrodes=dict(
                    type="array",
                    minItems=1,
                    items={"$ref": "#/properties/Ecephys/properties/definitions/Electrodes"}
                ),
                ElectricalSeries=dict(
                    type="array",
                    minItems=1,
                    items={"$ref": "#/properties/Ecephys/properties/definitions/ElectricalSeries"}
                )
            )
        )  
        
        return metadata_schema

    def get_metadata(self):

        # Extract information for specific parameters from .set file
        params_of_interest = [
            'experimenter',
            'comments',
            'duration',
            'sw_version'
        ]
        set_file = self.source_data['filename'].split('.')[0]+'.set'
        par = parse_generic_header(set_file, params_of_interest)

        # Extract information from AxonaRecordingExtractor
        elec_group_names = self.recording_extractor.get_channel_groups()
        unique_elec_group_names = set(elec_group_names)

        # Add available metadata
        metadata = super().get_metadata()
        metadata['NWBFile'] = dict(
            session_start_time=read_iso_datetime(set_file),
            session_description=par['comments'],
            #session_duration=par['duration']+'s',
            experimenter=[par['experimenter']]
        )

        metadata['Ecephys'] = dict(
            Device=[
                dict(
                    name="Axona",
                    description="Axona DacqUSB, sw_version={}"
                                .format(par['sw_version']),
                    manufacturer="Axona"
                ),
            ],
            ElectrodeGroup=[
                dict(
                    name=f'Group{group_name}',
                    location='',
                    device='Axona',
                    description=f"Group {group_name} electrodes.",
                )
                for group_name in unique_elec_group_names
            ],
            Electrodes=[
                dict(
                    name='group_name',
                    description="The name of the ElectrodeGroup this electrode is a part of.",
                    data=[f"Group{x}" for x in elec_group_names]
                )
            ],
            ElectricalSeries=dict(
                name='ElectricalSeries',
                description="Raw acquisition traces."
            )
        )

        return metadata

In [39]:
RXI = AxonaRecordingExtractorInterface(filename=filename)

In [40]:
RXI.get_metadata()

{'NWBFile': {'session_start_time': '2020-10-04T11:07:07',
  'session_description': '',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this electrode is a part of.',
    'data': ['Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group1',
     'Group1',
     'Group1',
     'Group1',
     'Group2',
     'Group2',
     'Group2',
     'Gr

In [41]:
base_metadata_schema = RXI.get_metadata_schema()

In [42]:
print(json.dumps(base_metadata_schema, indent=2))

{
  "required": [],
  "properties": {
    "Ecephys": {
      "required": [
        "Device",
        "ElectrodeGroup",
        "Electrodes",
        "ElectricalSeries"
      ],
      "properties": {
        "Device": {
          "type": "array",
          "minItems": 1,
          "items": {
            "$ref": "#/properties/Ecephys/properties/definitions/Device"
          }
        },
        "ElectrodeGroup": {
          "type": "array",
          "minItems": 1,
          "items": {
            "$ref": "#/properties/Ecephys/properties/definitions/ElectrodeGroup"
          }
        },
        "definitions": {
          "Device": {
            "required": [
              "name"
            ],
            "properties": {
              "name": {
                "description": "the name of this device",
                "type": "string"
              },
              "description": {
                "description": "Description of the device (e.g., model, firmware version, processing softwa

In [43]:
validate(
    instance=RXI.get_metadata(),
    schema=RXI.get_metadata_schema()
)

ValidationError: Additional properties are not allowed ('NWBFile' was unexpected)

Failed validating 'additionalProperties' in schema:
    {'$id': 'metadata.schema.json',
     '$schema': 'http://json-schema.org/draft-07/schema#',
     'additionalProperties': False,
     'description': 'Schema for the metadata',
     'properties': {'Ecephys': {'additionalProperties': False,
                                'properties': {'Device': {'items': {'$ref': '#/properties/Ecephys/properties/definitions/Device'},
                                                          'minItems': 1,
                                                          'type': 'array'},
                                               'ElectricalSeries': {'items': {'$ref': '#/properties/Ecephys/properties/definitions/ElectricalSeries'},
                                                                    'minItems': 1,
                                                                    'type': 'array'},
                        

In [ ]:
# The metadata_schema does not validate with the metadata, bc
# the full metadata_schema is only inherited when defining the
# nwbconverter (see below).

try:
    validate(
        instance=RXI.get_metadata(),
        schema=RXI.get_metadata_schema()
    )
    print('validation SUCCESS')
except ValidationError:
    print('validation FAILED')

In [25]:
from nwb_conversion_tools import (
    NWBConverter
)


class HussainiNWBConverter(NWBConverter):
    
    #@classmethod
    #def validate_source(cls, source_data):
    #    """Validate source_data against Converter source_schema."""
    #    validate(instance=source_data, schema=cls.get_source_schema())
    #    print('Source data is valid!')
    # 
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface
    )

In [26]:
validate(instance=source_data, schema=HussainiNWBConverter.get_source_schema())

In [27]:
# Get source_schema from converter

source_schema = HussainiNWBConverter.get_source_schema()
print(json.dumps(source_schema['properties'], indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "required": [
      "filename"
    ],
    "properties": {
      "filename": {
        "type": "string",
        "format": "file",
        "description": "Path to Axona files."
      }
    },
    "type": "object",
    "additionalProperties": true
  }
}


In [28]:
# Define source data (required for instantiating converter)

source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)

print(json.dumps(source_data, indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s"
  }
}


In [29]:
HussainiNWBConverter.data_interface_classes

{'AxonaRecordingExtractorInterface': __main__.AxonaRecordingExtractorInterface}

In [30]:
HussainiNWBConverter.data_interface_classes.items()

dict_items([('AxonaRecordingExtractorInterface', <class '__main__.AxonaRecordingExtractorInterface'>)])

In [31]:
validate(instance=source_data, schema=HussainiNWBConverter.get_source_schema())

In [32]:
HussainiNWBConverter.get_source_schema()

{'required': [],
 'properties': {'AxonaRecordingExtractorInterface': {'required': ['filename'],
   'properties': {'filename': {'type': 'string',
     'format': 'file',
     'description': 'Path to Axona files.'}},
   'type': 'object',
   'additionalProperties': True}},
 'type': 'object',
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'source.schema.json',
 'title': 'Source data schema',
 'description': 'Schema for the source data, files and directories',
 'version': '0.1.0'}

In [33]:
# Instantiate Hussaini-lab converter

HussainiNWBConverter.validate_source(source_data=source_data)

Source data is valid!


In [34]:
# Get metadata_schema from converter

converter = HussainiNWBConverter(source_data=source_data)
metadata_schema = converter.get_metadata_schema()
print(json.dumps(metadata_schema['properties'], indent=2))

Source data is valid!
{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": ""
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "3d449bb2-5025-4409-95da-e23f916ede29"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "2020-10-04T11:07:07"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment",
        "default": [
          "Abid"
     

In [35]:
metadata = converter.get_metadata()
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': '19724613-6a00-4542-a2d3-4fbeebfaecba',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}]}}

In [36]:
metadata['Ecephys']

{'Device': [{'name': 'Axona',
   'description': 'Axona DacqUSB, sw_version=1.2.2.16',
   'manufacturer': 'Axona'}],
 'ElectrodeGroup': [{'name': 'Group0',
   'location': '',
   'device': 'Axona',
   'description': 'Group 0 electrodes.'},
  {'name': 'Group1',
   'location': '',
   'device': 'Axona',
   'description': 'Group 1 electrodes.'},
  {'name': 'Group2',
   'location': '',
   'device': 'Axona',
   'description': 'Group 2 electrodes.'},
  {'name': 'Group3',
   'location': '',
   'device': 'Axona',
   'description': 'Group 3 electrodes.'}]}

In [51]:
# Validate metadata with metadata_schema from nwbconverter

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

ValidationError: Additional properties are not allowed ('session_duration' was unexpected)

Failed validating 'additionalProperties' in schema['properties']['NWBFile']:
    {'additionalProperties': False,
     'properties': {'data_collection': {'description': 'Notes about data '
                                                       'collection and '
                                                       'analysis.',
                                        'type': 'string'},
                    'experiment_description': {'description': 'general '
                                                              'description '
                                                              'of the '
                                                              'experiment',
                                               'type': 'string'},
                    'experimenter': {'default': ['Abid'],
                                     'description': 'name of person who '
                        

## Test axonadatainterface from nwb_conversion_tools

In [58]:
import json
from jsonschema import validate

In [59]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [60]:
from nwb_conversion_tools import axonadatainterface

In [61]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface
)

In [62]:
import sys
sys.path.append('/mnt/d/spikeinterface/hussaini-lab-to-nwb/hussaini_lab_to_nwb')

In [63]:
from hussaininwbconverter import HussainiNWBConverter

In [64]:
# Get source_schema from converter

source_schema = HussainiNWBConverter.get_source_schema()
print(json.dumps(source_schema['properties'], indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "required": [
      "filename"
    ],
    "properties": {
      "filename": {
        "type": "string",
        "format": "file",
        "description": "Path to Axona files."
      }
    },
    "type": "object",
    "additionalProperties": true
  }
}


In [65]:
source_data = dict(
    AxonaRecordingExtractorInterface=dict(
        filename=filename
    )
)
print(json.dumps(source_data, indent=2))

{
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s"
  }
}


In [66]:
# Validate source_data with source_schema

validate(
    instance=source_data,
    schema=source_schema
)

In [67]:
# Instantiate Hussaini-lab converter

converter = HussainiNWBConverter(source_data=source_data)

Source data is valid!


In [68]:
import datetime

In [69]:
# Get metadata_schema from converter
metadata = converter.get_metadata()
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': '80690991-d726-4d08-8386-1553fb90706c',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this electrode is a part of.',
    'data': ['Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group1',
     'Group1',
     'Group1',
     'Group1

In [70]:
get_schema_from_hdmf_class(ElectricalSeries)

{'required': ['name'],
 'properties': {'name': {'description': 'The name of this TimeSeries dataset',
   'type': 'string'},
  'resolution': {'description': 'The smallest meaningful difference (in specified unit) between values in data',
   'type': 'number',
   'default': -1.0},
  'conversion': {'description': 'Scalar to multiply each element in data to convert it to the specified unit',
   'type': 'number',
   'default': 1.0},
  'starting_time': {'description': 'The timestamp of the first sample',
   'type': 'number'},
  'rate': {'description': 'Sampling rate in Hz', 'type': 'number'},
  'comments': {'description': 'Human-readable comments about this TimeSeries dataset',
   'type': 'string',
   'default': 'no comments'},
  'description': {'description': 'Description of this TimeSeries dataset',
   'type': 'string',
   'default': 'no description'}},
 'type': 'object',
 'additionalProperties': False,
 'tag': 'pynwb.ecephys.ElectricalSeries'}

In [72]:
converter.get_metadata_schema()

KeyError: 'properties'

In [71]:
# Validate metadata with metadata_schema from nwbconverter

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

KeyError: 'properties'

In [28]:
print(json.dumps(converter.get_metadata_schema(), indent=2))

{
  "required": [
    "NWBFile"
  ],
  "properties": {
    "NWBFile": {
      "required": [
        "session_description",
        "identifier",
        "session_start_time"
      ],
      "properties": {
        "session_description": {
          "type": "string",
          "format": "long",
          "description": "a description of the session where this data was generated",
          "default": ""
        },
        "identifier": {
          "type": "string",
          "description": "a unique text identifier for the file",
          "default": "7305b39e-86cf-47c0-a88f-eb45d96aee67"
        },
        "session_start_time": {
          "type": "string",
          "description": "the start date and time of the recording session",
          "format": "date-time",
          "default": "2020-10-04T11:07:07"
        },
        "experimenter": {
          "type": "array",
          "items": {
            "type": "string",
            "title": "experimenter"
          },
          "descrip

In [29]:
# Test pytest function test_interface_schemas() for this specific interface

from jsonschema import Draft7Validator

In [30]:
data_interface = AxonaRecordingExtractorInterface(filename=filename)

schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

In [54]:
dir_name

'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'

In [56]:
converter.run_conversion(metadata=metadata, nwbfile_path=os.path.join(dir_name, 'axonadatainterface_test.nwb'))

UnsupportedOperation: Cannot read data from file /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axonadatainterface_test.nwb in mode 'r+'. There are no values.

In [52]:
from nwb_conversion_tools.conversion_tools import get_default_nwbfile_metadata
from nwb_conversion_tools.json_schema_utils import dict_deep_update

In [53]:
metadata = dict_deep_update(get_default_nwbfile_metadata(), metadata)

In [54]:
metadata['NWBFile']

{'session_description': '',
 'session_start_time': '2020-10-04T11:07:07',
 'identifier': '88fdeb9b-bb2b-43ce-a30f-f5901a1bff62',
 'experimenter': ['Abid']}

In [55]:
metadata_test = metadata

In [56]:
metadata_test

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': '88fdeb9b-bb2b-43ce-a30f-f5901a1bff62',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}]}}

In [57]:
converter.data_interface_objects

{'AxonaRecordingExtractorInterface': <__main__.AxonaRecordingExtractorInterface at 0x7ff3341401f0>}

In [58]:
metadata["NWBFile"]

{'session_description': '',
 'session_start_time': '2020-10-04T11:07:07',
 'identifier': '88fdeb9b-bb2b-43ce-a30f-f5901a1bff62',
 'experimenter': ['Abid']}

In [59]:
from datetime import datetime
from pynwb import NWBFile

In [60]:
def make_nwbfile_from_metadata(metadata: dict):
    """Make NWBFile from available metadata."""
    metadata = dict_deep_update(get_default_nwbfile_metadata(), metadata)
    nwbfile_kwargs = metadata["NWBFile"]
    if "Subject" in metadata:
        # convert ISO 8601 string to datetime
        if "date_of_birth" in metadata["Subject"] and isinstance(metadata["Subject"]["date_of_birth"], str):
            metadata["Subject"]["date_of_birth"] = datetime.fromisoformat(metadata["Subject"]["date_of_birth"])
        nwbfile_kwargs.update(subject=Subject(**metadata["Subject"]))
    # convert ISO 8601 string to datetime
    if isinstance(nwbfile_kwargs.get("session_start_time", None), str):
        nwbfile_kwargs["session_start_time"] = datetime.fromisoformat(metadata["NWBFile"]["session_start_time"])
    return NWBFile(**nwbfile_kwargs)

In [61]:
nwbfile = make_nwbfile_from_metadata(metadata=metadata_test)

/mnt/d/spikeinterface/hussaini-to-nwb/venv/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [62]:
print(type(metadata_test['Ecephys']['Device']))
print(type(metadata_test['Ecephys']['Device'][0]))

<class 'list'>
<class 'dict'>


In [63]:
import spikeextractors as se
import numpy as np

In [64]:
r_cache = se.load_extractor_from_pickle(os.path.join(dir_name, 'cached_data_preproc.pkl'))

In [65]:
data_interface = HussainiNWBConverter

In [66]:
metadata_test

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': '88fdeb9b-bb2b-43ce-a30f-f5901a1bff62',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}]}}

In [67]:
converter.run_conversion(metadata=metadata_test,
                         save_to_file=False,
                         nwbfile_path=None,
                         nwbfile=None)

Using Device to instantiate electrode group


/mnt/d/spikeinterface/hussaini-to-nwb/venv/lib/python3.8/site-packages/hdmf/common/table.py:415: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)


root pynwb.file.NWBFile at 0x140682527782128
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 4, 27, 21, 7, 59, 871506, tzinfo=tzlocal())]
  identifier: 88fdeb9b-bb2b-43ce-a30f-f5901a1bff62
  session_start_time: 2020-10-04 11:07:07-04:00
  timestamps_reference_time: 2020-10-04 11:07:07-04:00

In [69]:
import datetime

In [70]:
metadata=converter.get_metadata()
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': 'fd4d9661-9fc2-4532-b38e-7ed81eae2f64',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}]}}

In [71]:
# Get metadata_schema from converter
converter.run_conversion(metadata=converter.get_metadata(),
                         save_to_file=False,
                         nwbfile_path=None,
                         nwbfile=None)

Using Device to instantiate electrode group


root pynwb.file.NWBFile at 0x140682527810752
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 4, 27, 21, 8, 23, 515216, tzinfo=tzlocal())]
  identifier: 5cbd738e-54bc-4040-bef6-7cc468974b68
  session_start_time: 2020-10-04 11:07:07-04:00
  timestamps_reference_time: 2020-10-04 11:07:07-04:00

In [50]:
se.NwbRecordingExtractor.add_devices(
    recording=recording,
    nwbfile=nwbfile,
    metadata=metadata
)

In [79]:
metadata['Ecephys']['Device']

[{'name': 'Device', 'description': 'no description'}]

In [89]:
nwbfile.devices

{'Axona': Axona pynwb.device.Device at 0x140012976895456
 Fields:
   description: Axona DacqUSB, sw_version=1.2.2.16
   manufacturer: Axona,
 'Device': Device pynwb.device.Device at 0x140012976897712
 Fields:
   description: no description}

In [90]:
if nwbfile is not None:
    assert isinstance(nwbfile, NWBFile), "'nwbfile' should be of type pynwb.NWBFile"
if len(nwbfile.devices) == 0:
    se.NwbRecordingExtractor.add_devices(recording, nwbfile)
defaults = dict(
    name="Electrode Group",
    description="no description",
    location="unknown",
    device_name="Device"
)
if metadata is None or 'ElectrodeGroup' not in metadata['Ecephys']:
    metadata = dict(
        Ecephys=dict(
            ElectrodeGroup=[defaults]
        )
    )

In [91]:
assert all([isinstance(x, dict) for x in metadata['Ecephys']['ElectrodeGroup']]), \
    "Expected metadata['Ecephys']['ElectrodeGroup'] to be a list of dictionaries!"

In [94]:
defaults['device_name']

'Device'

In [95]:
metadata['Ecephys']['ElectrodeGroup']

[{'name': 'Group0',
  'location': '',
  'device': 'Axona',
  'description': 'Group 0 electrodes.'},
 {'name': 'Group1',
  'location': '',
  'device': 'Axona',
  'description': 'Group 1 electrodes.'},
 {'name': 'Group2',
  'location': '',
  'device': 'Axona',
  'description': 'Group 2 electrodes.'},
 {'name': 'Group3',
  'location': '',
  'device': 'Axona',
  'description': 'Group 3 electrodes.'}]

In [92]:
for grp in metadata['Ecephys']['ElectrodeGroup']:
    device_name = grp.get('device_name', defaults['device_name'])
    print(device_name)

Device
Device
Device
Device


In [93]:
metadata['Ecephys']['ElectrodeGroup']

[{'name': 'Group0',
  'location': '',
  'device': 'Axona',
  'description': 'Group 0 electrodes.'},
 {'name': 'Group1',
  'location': '',
  'device': 'Axona',
  'description': 'Group 1 electrodes.'},
 {'name': 'Group2',
  'location': '',
  'device': 'Axona',
  'description': 'Group 2 electrodes.'},
 {'name': 'Group3',
  'location': '',
  'device': 'Axona',
  'description': 'Group 3 electrodes.'}]

In [57]:
device_name

'Device'

In [58]:
metadata['Ecephys']['ElectrodeGroup']

[{'name': 'Group0',
  'location': '',
  'device': 'Axona',
  'description': 'Group 0 electrodes.'}]

In [62]:
grp = metadata['Ecephys']['ElectrodeGroup'][0]
grp

{'name': 'Group0',
 'location': '',
 'device': 'Axona',
 'description': 'Group 0 electrodes.'}

In [61]:
device_name = grp.get('device_name', defaults['device_name'])
device_name

'Device'

In [66]:
nwbfile.devices

{'Axona': Axona pynwb.device.Device at 0x140690185518672
 Fields:
   description: Axona DacqUSB, sw_version=1.2.2.16
   manufacturer: Axona}

In [64]:
nwbfile.electrode_groups

{}

In [63]:
grp.get('name', defaults['name']) not in nwbfile.electrode_groups

True

In [70]:
grp

{'name': 'Group0',
 'location': '',
 'device': 'Axona',
 'description': 'Group 0 electrodes.'}

In [68]:
nwbfile.devices

{'Axona': Axona pynwb.device.Device at 0x140690185518672
 Fields:
   description: Axona DacqUSB, sw_version=1.2.2.16
   manufacturer: Axona}

In [71]:
nwbfile.electrode_groups

{}

In [67]:
for grp in metadata['Ecephys']['ElectrodeGroup']:
    device_name = grp.get('device_name', defaults['device_name'])
    if grp.get('name', defaults['name']) not in nwbfile.electrode_groups:
        if device_name not in nwbfile.devices:
            new_device = dict(
                Ecephys=dict(
                    Device=dict(
                        name=device_name
                    )
                )
            )
            se.NwbRecordingExtractor.add_devices(recording, nwbfile, metadata=new_device)

AssertionError: Expected metadata['Ecephys']['Device'] to be a list of dictionaries!

In [236]:
len(nwbfile.devices)

1

In [237]:
se.NwbRecordingExtractor.add_electrode_groups(
    recording=recording,
    nwbfile=nwbfile,
    metadata=metadata
)

In [ ]:
converter.run_conversion(metadata=metadata_test,
                         save_to_file=False,
                         nwbfile_path=os.path.join(dir_name, 'axonadatainterface_test.nwb'),
                         nwbfile=None)

In [239]:
metadata_test

{'NWBFile': {'session_description': '',
  'session_start_time': '2020-10-04T11:07:07',
  'identifier': 'bff4a978-7d4d-4878-9c3b-c4c9a0a28e02',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this                         electrode is a part of.',
    'data': ['Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0',
     'Group0']}],
  'ElectricalSeries': {'name': 'ElectricalSeries',
   'description': 'Raw acquisition traces.'}}}

In [18]:
import nwb_conversion_tools

In [19]:
# Try to see for which data_interface pytest crashes
# Only for tutorialinterfaces! But it works fine for AxonaRecordingExtractorInterface!

interfaces_that_crash = [nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialRecordingInterface,
                         nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialSortingInterface]

from nwb_conversion_tools import interface_list

for data_interface in interface_list:
    
    if not data_interface in interfaces_that_crash:
    
        # check validity of source schema
        schema = data_interface.get_source_schema()
        Draft7Validator.check_schema(schema)

        # check validity of conversion options schema
        schema = data_interface.get_conversion_options_schema()
        Draft7Validator.check_schema(schema)

In [20]:
interfaces_that_crash = [nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialRecordingInterface,
                         nwb_conversion_tools.datainterfaces.tutorialdatainterface.TutorialSortingInterface]

data_interface = interfaces_that_crash[0]
        
# check validity of source schema
schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)

In [21]:
data_interface = interfaces_that_crash[1]
        
# check validity of source schema
schema = data_interface.get_source_schema()
Draft7Validator.check_schema(schema)

# check validity of conversion options schema
schema = data_interface.get_conversion_options_schema()
Draft7Validator.check_schema(schema)